In [1]:
!pip install -qq convokit

In [2]:
!pip install -qq scikit-surprise

IMPORTING NECESSARY LIBRARIES

In [3]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import convokit

MOVIE DIALOGUE CORPUS IS LOADED

In [4]:
movie_corpus = convokit.Corpus(filename=convokit.download("movie-corpus"))

PRINT BASIS STATS

In [5]:
print("Number of conversations:", len(movie_corpus.conversations))
print("Number of users:", len(movie_corpus.speakers))
print("Number of utterances:", len(movie_corpus.utterances))

Number of conversations: 83097
Number of users: 9035
Number of utterances: 304713


INFORMATION ON EACH CONVERSATION

In [6]:
for conv_id in movie_corpus.get_conversation_ids():
    conv = movie_corpus.get_conversation(conv_id)
    print("Conversation ID:", conv_id)
    print("Metadata:", conv.meta)
    print("Frequency of utterances in conversation:", len(conv.get_utterance_ids()))
    print()

Streaming output truncated to the last 5000 lines.
Conversation ID: L290355
Metadata: ConvoKitMeta({'movie_idx': 'm92', 'movie_name': 'house of 1000 corpses', 'release_year': '2003', 'rating': '5.60', 'votes': '27033', 'genre': "['horror']"})
Frequency of utterances in conversation: 3

Conversation ID: L290324
Metadata: ConvoKitMeta({'movie_idx': 'm92', 'movie_name': 'house of 1000 corpses', 'release_year': '2003', 'rating': '5.60', 'votes': '27033', 'genre': "['horror']"})
Frequency of utterances in conversation: 2

Conversation ID: L290308
Metadata: ConvoKitMeta({'movie_idx': 'm92', 'movie_name': 'house of 1000 corpses', 'release_year': '2003', 'rating': '5.60', 'votes': '27033', 'genre': "['horror']"})
Frequency of utterances in conversation: 5

Conversation ID: L290305
Metadata: ConvoKitMeta({'movie_idx': 'm92', 'movie_name': 'house of 1000 corpses', 'release_year': '2003', 'rating': '5.60', 'votes': '27033', 'genre': "['horror']"})
Frequency of utterances in conversation: 3

Conve

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
Conversation ID: L655326
Metadata: ConvoKitMeta({'movie_idx': 'm606', 'movie_name': 'wild things', 'release_year': '1998', 'rating': '6.60', 'votes': '40523', 'genre': "['crime', 'mystery', 'thriller']"})
Frequency of utterances in conversation: 2

Conversation ID: L655322
Metadata: ConvoKitMeta({'movie_idx': 'm606', 'movie_name': 'wild things', 'release_year': '1998', 'rating': '6.60', 'votes': '40523', 'genre': "['crime', 'mystery', 'thriller']"})
Frequency of utterances in conversation: 4

Conversation ID: L655320
Metadata: ConvoKitMeta({'movie_idx': 'm606', 'movie_name': 'wild things', 'release_year': '1998', 'rating': '6.60', 'votes': '40523', 'genre': "['crime', 'mystery', 'thriller']"})
Frequency of utterances in conversation: 2

Conversation ID: L654917
Metadata: ConvoKitMeta({'movie_idx': 'm606', 'movie_name': 'wild things', 'release_year': '1998', 'rating': '6.60', 'votes': '40523', 'genre': "['crime', 'mystery', 'thriller']"

DATA ITERATED IS STORED INTO EMPTY LISTS

In [8]:
conversation_ids = []
movie_indices = []
movie_names = []
release_years = []
ratings = []
votes = []
genres = []
num_utterances = []

# Loop through conversations and extract data
for conv_id in movie_corpus.get_conversation_ids():
    convo = movie_corpus.get_conversation(conv_id)

    # Extract metadata from ConvoKitMeta object
    metadata = convo.meta

    # Append data to respective lists
    conversation_ids.append(conv_id)
    movie_indices.append(metadata['movie_idx'])
    movie_names.append(metadata['movie_name'])
    release_years.append(metadata['release_year'])
    ratings.append(metadata['rating'])
    votes.append(metadata['votes'])
    genres.append(metadata['genre'])
    num_utterances.append(len(convo.get_utterance_ids()))

# Create a DataFrame from the lists
data = {
    'Conversation ID': conversation_ids,
    'Movie Index': movie_indices,
    'Movie Name': movie_names,
    'Rating': ratings,
}

data = pd.DataFrame(data)

In [9]:
data

,Conversation ID,Movie Index,Movie Name,Rating
0,L1044,m0,10 things i hate about you,6.90
1,L984,m0,10 things i hate about you,6.90
2,L924,m0,10 things i hate about you,6.90
3,L870,m0,10 things i hate about you,6.90
4,L866,m0,10 things i hate about you,6.90
...,...,...,...,...
83092,L666324,m616,zulu dawn,6.40
83093,L666262,m616,zulu dawn,6.40
83094,L666520,m616,zulu dawn,6.40
83095,L666369,m616,zulu dawn,6.40


In [10]:
reader = Reader(rating_scale=(1, 10))

In [15]:
data = df[['Conversation ID','Movie Name','Rating']]
data = Dataset.load_from_df(data, reader)


# Split the data into training and testing sets (80% train, 20% test)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# FITTING THE MODEL
model = SVD()
model.fit(trainset)

# MODEL EVALUATION
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)

RMSE: 0.0641


In [20]:
# Generate recommendations for a specific user (e.g., user_id='L236416')
user_id = 'L36547'
user_movies = df[df['Conversation ID'] == user_id]['Movie Name'].unique()

# A LIST OF UNRATED MOVIES IS CREATED
all_movies = df['Movie Name'].unique()
unrated_movies = np.setdiff1d(all_movies, user_movies)

# RATING UNRATED MOVIES
user_recommendations = []
for movie_id in unrated_movies:
    predicted_rating = model.predict(user_id, movie_id).est
    user_recommendations.append((movie_id, predicted_rating))

    # SORTING THE RECOMMENTATIONS
user_recommendations.sort(key=lambda x: x[1], reverse=True)

In [23]:
top_movies = 3
print(f'Top {top_movies} recommendations for user {user_id}:')
for movie_id, predicted_rating in user_recommendations[:top_movies]:
    print(f'MOVIE: {movie_id}, Rating(PREDICTED): {predicted_rating:.2f}')

Top 3 recommendations for user L36547:
MOVIE: neuromancer, Rating(PREDICTED): 9.25
MOVIE: the godfather, Rating(PREDICTED): 9.13
MOVIE: the godfather: part ii, Rating(PREDICTED): 8.90
